In [1]:
import numpy as np
import os
import imutils
import dlib
import cv2

import imageio
from imutils import face_utils

import os

import pickle

In [2]:
people = ['F01','F02','F04','F05','F06','F07','F08','F09','F10','F11','M01','M02','M04','M07','M08']
folder_enum = ['01','02','03','04','05','06','07','08', '09', '10']
instances = ['01','02','03','04','05','06','07','08', '09', '10']
data_types = ['words']
words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']

max_seq_length = 22

X_train = []
y_train = []
X_val = []
y_val = []
X_test = []
y_test = []

MAX_WIDTH = 100
MAX_HEIGHT = 100


In [3]:
from skimage.transform import resize
import time

t1 = time.time()
UNSEEN_VALIDATION_SPLIT = ['09']
UNSEEN_TEST_SPLIT = ['05']
directory = "cropped"

for person_id in people:
    tx1 = time.time()
    for data_type in data_types:
        for word_index, word in enumerate(folder_enum):
            for iteration in instances:
                path = os.path.join(directory, person_id, data_type, word, iteration)
                filelist = sorted(os.listdir(path + '/'))
                sequence = [] 
                for img_name in filelist:
                    if img_name.startswith('color'):
                        image = imageio.imread(path + '/' + img_name)
                        image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                        image = 255 * image
                        # Convert to integer data type pixels.
                        image = image.astype(np.uint8)
                        sequence.append(image)                        
                pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]                            
                sequence.extend(pad_array * (max_seq_length - len(sequence)))
                sequence = np.array(sequence)
                                
                if  iteration in UNSEEN_TEST_SPLIT:
                    X_test.append(sequence)
                    y_test.append(word_index)
                elif iteration in UNSEEN_VALIDATION_SPLIT:
                    X_val.append(sequence)
                    y_val.append(word_index)
                else:
                    X_train.append(sequence)
                    y_train.append(word_index)    
    tx2 = time.time()
    print(f'Finished reading images for person {person_id}. Time taken : {tx2 - tx1} secs.')    
    
t2 = time.time()
print(f"Time taken for creating constant size 3D Tensors from those cropped lip regions : {t2 - t1} secs.")


Finished reading images for person F01. Time taken : 7.718793630599976 secs.
Finished reading images for person F02. Time taken : 6.943119049072266 secs.
Finished reading images for person F04. Time taken : 9.41447377204895 secs.
Finished reading images for person F05. Time taken : 8.225896835327148 secs.
Finished reading images for person F06. Time taken : 8.804054975509644 secs.
Finished reading images for person F07. Time taken : 11.754110097885132 secs.
Finished reading images for person F08. Time taken : 6.600075006484985 secs.
Finished reading images for person F09. Time taken : 5.931450605392456 secs.
Finished reading images for person F10. Time taken : 5.035379648208618 secs.
Finished reading images for person F11. Time taken : 5.054295301437378 secs.
Finished reading images for person M01. Time taken : 6.51457405090332 secs.
Finished reading images for person M02. Time taken : 7.78283953666687 secs.
Finished reading images for person M04. Time taken : 7.251882076263428 secs.
F

In [4]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

# X_train = X_train.tolist()
# X_val = X_val.tolist()
# X_test = X_test.tolist()

len(X_train)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# y_train = y_train.tolist()
# y_val = y_val.tolist()
# y_test = y_test.tolist()

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)


(1200, 22, 100, 100)
(150, 22, 100, 100)
(150, 22, 100, 100)
(1200,)
(150,)
(150,)


In [7]:
y_train.shape

(1200,)

In [8]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = y_train.reshape(len(y_train), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [9]:
onehot_encoded.shape

(1200, 10)

In [10]:
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    X = (X - v_min)/(v_max - v_min)
    X = np.nan_to_num(X)
    return X

from keras.utils import np_utils, generic_utils

X_train = normalize_it(X_train)
X_val = normalize_it(X_val)
X_test = normalize_it(X_test)

y_train = y_train.reshape(len(y_train), 1)
y_train = onehot_encoder.fit_transform(y_train)

y_test = y_test.reshape(len(y_test), 1)
y_test = onehot_encoder.fit_transform(y_test)

y_val = y_val.reshape(len(y_val), 1)
y_val = onehot_encoder.fit_transform(y_val)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4)
X_test = np.expand_dims(X_test, axis=4)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

import pickle

with open('pickle_train_test_ohe/X_train.pkl', 'wb') as f:
  pickle.dump(X_train, f)

with open('pickle_train_test_ohe/y_train.pkl', 'wb') as f:
  pickle.dump(y_train, f)

with open('pickle_train_test_ohe/X_test.pkl', 'wb') as f:
  pickle.dump(X_test, f)

with open('pickle_train_test_ohe/y_test.pkl', 'wb') as f:
  pickle.dump(y_test, f)

with open('pickle_train_test_ohe/X_val.pkl', 'wb') as f:
  pickle.dump(X_val, f)

with open('pickle_train_test_ohe/y_val.pkl', 'wb') as f:
  pickle.dump(y_val, f)

<ipython-input-10-6033e1c24a39>:4: RuntimeWarning: invalid value encountered in true_divide
  X = (X - v_min)/(v_max - v_min)


(1200, 22, 100, 100, 1)
(150, 22, 100, 100, 1)
(150, 22, 100, 100, 1)


USING NP UTILS

In [ ]:

"""def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    X = (X - v_min)/(v_max - v_min)
    X = np.nan_to_num(X)
    return X

from keras.utils import np_utils, generic_utils

X_train = normalize_it(X_train)
X_val = normalize_it(X_val)
X_test = normalize_it(X_test)

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
y_val = np_utils.to_categorical(y_val, 10)

from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4)
X_test = np.expand_dims(X_test, axis=4)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

import pickle

with open('pickle_train_test/X_train.pkl', 'wb') as f:
  pickle.dump(X_train, f)

with open('pickle_train_test/y_train.pkl', 'wb') as f:
  pickle.dump(y_train, f)

with open('pickle_train_test/X_test.pkl', 'wb') as f:
  pickle.dump(X_test, f)

with open('pickle_train_test/y_test.pkl', 'wb') as f:
  pickle.dump(y_test, f)

with open('pickle_train_test/X_val.pkl', 'wb') as f:
  pickle.dump(X_val, f)

with open('pickle_train_test/y_val.pkl', 'wb') as f:
  pickle.dump(y_val, f)"""